In [36]:
import pandas as pd

df = pd.read_csv("../data/processed/clean_store.csv")
df.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


In [37]:
date_dim = pd.DataFrame()

date_dim['date_key'] = pd.to_datetime(df['Order Date']).dt.strftime('%Y%m%d')
date_dim['date'] = pd.to_datetime(df['Order Date'])
date_dim['year'] = date_dim['date'].dt.year
date_dim['quarter'] = date_dim['date'].dt.quarter
date_dim['month'] = date_dim['date'].dt.month
date_dim['month_name'] = date_dim['date'].dt.month_name()
date_dim['day'] = date_dim['date'].dt.day
date_dim['day_of_week'] = date_dim['date'].dt.day_of_week
date_dim['day_name'] = date_dim['date'].dt.day_name()

weekend_map = {
    'Monday': 0,
    'Tuesday': 0,
    'Wednesday': 0,
    'Thursday': 0,
    'Friday': 0,
    'Saturday': 1,
    'Sunday': 1,
}

date_dim['is_weekend'] = date_dim['day_name'].map(weekend_map)
date_dim['year_month'] = date_dim['date'].dt.strftime('%Y-%m')

date_dim = date_dim.drop_duplicates("date")

date_dim.head()

,date_key,date,year,quarter,month,month_name,day,day_of_week,day_name,is_weekend,year_month
0,20161108,2016-11-08,2016,4,11,November,8,1,Tuesday,0,2016-11
2,20160612,2016-06-12,2016,2,6,June,12,6,Sunday,1,2016-06
3,20151011,2015-10-11,2015,4,10,October,11,6,Sunday,1,2015-10
5,20140609,2014-06-09,2014,2,6,June,9,0,Monday,0,2014-06
12,20170415,2017-04-15,2017,2,4,April,15,5,Saturday,1,2017-04


In [38]:
customer_dim = pd.DataFrame()

customer_dim['Customer ID'] = df['Customer ID']
customer_dim['Customer Name'] = df['Customer Name']
customer_dim['Segment'] = df['Segment']
customer_dim['Country'] = df['Country']
customer_dim['State'] = df['State']
customer_dim['City'] = df['City']
customer_dim['Postal Code'] = df['Postal Code']
customer_dim['Region'] = df['Region']

customer_dim = customer_dim.drop_duplicates("Customer ID")

customer_dim.head()

,Customer ID,Customer Name,Segment,Country,State,City,Postal Code,Region
0,CG-12520,Claire Gute,Consumer,United States,Kentucky,Henderson,42420,South
2,DV-13045,Darrin Van Huff,Corporate,United States,California,Los Angeles,90036,West
3,SO-20335,Sean O'Donnell,Consumer,United States,Florida,Fort Lauderdale,33311,South
5,BH-11710,Brosina Hoffman,Consumer,United States,California,Los Angeles,90032,West
12,AA-10480,Andrew Allen,Consumer,United States,North Carolina,Concord,28027,South


In [39]:
product_dim = pd.DataFrame()

product_dim['Product ID'] = df['Product ID']
product_dim['Product Name'] = df['Product Name']
product_dim['Sub-Category'] = df['Sub-Category']
product_dim['Category'] = df['Category']

product_dim = product_dim.drop_duplicates("Product ID")

product_dim.head()

,Product ID,Product Name,Sub-Category,Category
0,FUR-BO-10001798,Bush Somerset Collection Bookcase,Bookcases,Furniture
1,FUR-CH-10000454,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",Chairs,Furniture
2,OFF-LA-10000240,Self-Adhesive Address Labels for Typewriters b...,Labels,Office Supplies
3,FUR-TA-10000577,Bretford CR4500 Series Slim Rectangular Table,Tables,Furniture
4,OFF-ST-10000760,Eldon Fold 'N Roll Cart System,Storage,Office Supplies


In [40]:
fact_orders = df[['Order ID', 'Customer ID', 'Product ID', 'Sales', 'Profit', 'Quantity', 'Discount']].copy()

fact_orders['date_key'] = pd.to_datetime(df['Order Date']).dt.strftime('%Y%m%d')

fact_orders.head()

,Order ID,Customer ID,Product ID,Sales,Profit,Quantity,Discount,date_key
0,CA-2016-152156,CG-12520,FUR-BO-10001798,261.9600,41.9136,2,0.00,20161108
1,CA-2016-152156,CG-12520,FUR-CH-10000454,731.9400,219.5820,3,0.00,20161108
2,CA-2016-138688,DV-13045,OFF-LA-10000240,14.6200,6.8714,2,0.00,20160612
3,US-2015-108966,SO-20335,FUR-TA-10000577,957.5775,-383.0310,5,0.45,20151011
4,US-2015-108966,SO-20335,OFF-ST-10000760,22.3680,2.5164,2,0.20,20151011


In [ ]:
date_check = fact_orders['date_key'].isin(date_dim['date_key']).all()
customer_check = fact_orders['Customer ID'].isin(customer_dim['Customer ID']).all()
product_check = fact_orders['Product ID'].isin(product_dim['Product ID']).all()

print(date_check, product_check, customer_check)

True True True


In [41]:
date_dim.to_csv("../data/processed/date_dim.csv", index = False)
customer_dim.to_csv("../data/processed/customer_dim.csv", index = False)
product_dim.to_csv("../data/processed/product_dim.csv", index = False)
fact_orders.to_csv("../data/processed/fact_orders.csv", index = False)